In [2]:
import util
from test_framework.address import program_to_witness
from test_framework.key import generate_key_pair
from test_framework.messages import COutPoint, CTransaction, CTxIn, CTxOut, CTxInWitness
from test_framework.script import SegwitV0SignatureHash, SIGHASH_ALL, hash160, get_p2pkh_script

privkey, pubkey = generate_key_pair()
program=hash160(pubkey.get_bytes(bip340=False))
version=0x00
address=program_to_witness(version,program)
test=util.TestWrapper()
test.setup()
node=test.nodes[0]
version=node.getnetworkinfo()['subversion']
node.generate(101)
balance=node.getbalance()
assert balance>1

TestWrapper is already running!


In [6]:
unspent_txid=node.listunspent(1)[-1]["txid"]
inputs = [{"txid": unspent_txid, "vout": 0}]
tx_hex=node.createrawtransaction(inputs=inputs,outputs=[{address:1}])
res = node.signrawtransactionwithwallet(hexstring=tx_hex)
tx_hex = res["hex"]
assert res["complete"]
assert 'errors' not in res

txid = node.sendrawtransaction(hexstring=tx_hex, maxfeerate=0)

spending_tx = CTransaction()
spending_tx.nVersion = 1
spending_tx.nLockTime = 0
outpoint = COutPoint(int(txid, 16), 0)
spending_tx_in = CTxIn(outpoint)
spending_tx.vin = [spending_tx_in]
#generate new bitcoin core wallet address
dest_addr = node.getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(node.getaddressinfo(dest_addr)['scriptPubKey'])
amount_sat = int(0.5 * 100_000_000)
dest_output = CTxOut(amount_sat, scriptpubkey)
spending_tx.vout = [dest_output]
# generate the sigwit v0 signature hash for signing
sighash = SegwitV0SignatureHash(script=get_p2pkh_script(program),txTo=spending_tx,inIdx=0, hashtype=SIGHASH_ALL, amount=100_000_000)
sig = privkey.sign_ecdsa(sighash) + chr(SIGHASH_ALL).encode('latin-1')
spending_tx.wit.vtxinwit.append(CTxInWitness([sig,pubkey.get_bytes(bip340=False)]))
# serialize signed transaction for broadcast
spending_tx_str=spending_tx.serialize().hex()
assert node.testmempoolaccept(rawtxs=[spending_tx_str], maxfeerate=0)[0]['allowed']
test.shutdown()

2021-11-26T12:03:13.030000Z TestFramework./var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_p88vu6gr (INFO): Stopping nodes
2021-11-26T12:03:13.505000Z TestFramework./var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_p88vu6gr (INFO): Cleaning up /var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_p88vu6gr on exit
2021-11-26T12:03:13.506000Z TestFramework./var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_p88vu6gr (INFO): Tests successful


In [9]:
# simple way
test = util.TestWrapper()
test.setup()
node = test.nodes[0]
tx=node.generate_and_send_coins(address)
spending_tx=test.create_spending_transaction(tx.hash)
sighash=SegwitV0SignatureHash(script=get_p2pkh_script(program), txTo=spending_tx,inIdx=0,hashtype=SIGHASH_ALL,amount=100_000_000)
sig=privkey.sign_ecdsa(sighash) + chr(SIGHASH_ALL).encode('latin-1')
spending_tx.wit.vtxinwit.append(CTxInWitness([sig,pubkey.get_bytes(bip340=False)]))
test.shutdown()

TestWrapper is already running!

Client version is /Satoshi:22.99.0(testnode0)/

Balance: 50.00000000

2021-11-26T12:07:32.671000Z TestFramework./var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_vmjcym90 (INFO): Stopping nodes
2021-11-26T12:07:33.102000Z TestFramework./var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_vmjcym90 (INFO): Cleaning up /var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_vmjcym90 on exit
2021-11-26T12:07:33.103000Z TestFramework./var/folders/cj/pzq3pqcn6sx9gkvw0pfbpmvh0000gn/T/bitcoin_func_test_vmjcym90 (INFO): Tests successful
